In [1]:
import sys
sys.path.append('..')

from sqlalchemy import func, text, cast, String

import webbrowser

from src import crud
from src.database.session import db
from src.services import facebook_image_descriptive_saver as fi, FacebookImageDescriptiveDF
from src.pingers import *
from src.models.enums.facebook import CreativeType
from src.fb_api import get_preview_shareable_link
shop_id = 16038

2023-11-29 09:55:01.346 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [2]:
creative_type = CreativeType.image
fcf_subquery = (
    db.query(FacebookCreativeFeatures.ad_id, FacebookCreativeFeatures.shop_id)
    .filter(func.random() < 0.001, FacebookCreativeFeatures.creative_type == creative_type)
    .limit(100)
).subquery()
creatives_query = (
    db.query(FacebookAd.shop_id, FacebookAd.ad_id, FacebookAd.creative, FacebookAd.image_ad_image_hash)
    .join(
        fcf_subquery, (fcf_subquery.c.ad_id == FacebookAd.ad_id) & (fcf_subquery.c.shop_id == FacebookAd.shop_id)
    )
)
creatives_df = pd.read_sql(creatives_query.statement, db.bind)

# credentials_dict = {
#     shop_id: crud.credentials.get_facebook_access_token_by_shop(db=db, shop_id=int(shop_id))
#     for shop_id in creatives_df.shop_id.astype(int).unique()
# }
# creatives_df = creatives_df[
#     creatives_df.shop_id.isin(
#         [shop_id for shop_id in credentials_dict if credentials_dict[shop_id] is not None]
#     )
# ]
# logger.debug(len(creatives_df))
# creatives_df["preview_shareable_link"] = creatives_df.apply(
#     lambda s: get_preview_shareable_link(
#         ad_id=s.ad_id,
#         access_token=credentials_dict[s.shop_id],
#     ),
#     axis=1,
# )
# creatives_df = creatives_df[creatives_df.preview_shareable_link.notna()]
creatives_df

2023-11-29 09:55:03.745 | DEBUG    | src.models.facebook.facebook_ad:image_ad_image_hash:33 - expression
2023-11-29 09:55:06.446 | DEBUG    | src.models.facebook.facebook_ad:image_ad_image_hash:33 - expression
2023-11-29 09:55:06.737 | DEBUG    | src.models.facebook.facebook_ad:image_ad_image_hash:33 - expression


,shop_id,ad_id,creative,image_ad_image_hash
0,15873,6046992849236,"{'id': '6046798888236', 'body': 'Exclusive bot...",9e766a20b1c3065f92c1bb7a0d0dc7b9
1,15873,6047054592836,"{'id': '6047159428236', 'body': 'Exclusive bot...",f9f6fc2a69d48069134c512df4843580
2,15873,6047064018436,"{'id': '6047159483436', 'body': 'Exclusive bot...",08b528fd9daf02ac8fa16c2cd3ecaef6
3,15873,6047603661636,"{'id': '6047158306636', 'body': 'Exclusive bot...",9e766a20b1c3065f92c1bb7a0d0dc7b9
4,134490,6066828917334,"{'id': '6066828900534', 'body': 'Build a perso...",dc5f07ca8606b69e56d2df81864b66ea
...,...,...,...,...
95,38620545,6154330509453,"{'id': '6154046906653', 'body': '😍 Der letzte ...",43f542c93554230c0a374751296fb09d
96,38620545,6154939328453,"{'id': '6151870931853', 'body': '💙 Die perfekt...",fd2f2f4065452ed94796b6821d8159d8
97,38620545,6158005023653,"{'id': '6157869352853', 'body': '💸 Beginne sch...",83733daae0a8747442d3ebf6c040c579
98,38620545,6159764713053,"{'id': '6159242335853', 'body': 'Sichere dir 2...",3fb4947359fa58b7f1a93f1dda34fca0


In [4]:
failed = creatives_df[creatives_df.image_ad_image_hash.isna()]

In [12]:
row = failed.sample(1)
print(json.dumps(row.creative.item(), indent=4))
# webbrowser.open(row.preview_shareable_link)
# image_url = db.query(Image).filter(Image.image_hash == row)

{
    "id": "23845659961350526",
    "asset_feed_spec": {
        "bodies": [
            {
                "text": "\"I use the Light Serum on my scalp and in my ears, now my psoriasis is GONE and I can finally stop itching.\"  - @ohmichellelee \ud83d\udc9b",
                "adlabels": [
                    {
                        "id": "23844998022450526",
                        "name": "placement_asset_f1c0ad569c31d7c_1593652674026"
                    }
                ]
            }
        ],
        "images": [
            {
                "hash": "1769f86c59d1a0261ec75647e663e112",
                "adlabels": [
                    {
                        "id": "23844998022380526",
                        "name": "placement_asset_f3ceb06a862749c_1593652674025"
                    }
                ]
            }
        ],
        "titles": [
            {
                "text": "Bodewell - Skincare for the rest of us",
                "adlabels": [
                   

In [2]:
crud.credentials.get_facebook_access_token_by_shop(db=db)

In [2]:
fi.add_global_centroids(force_from_scratch=True)
fi.create_and_save_main()
fi.create_and_save_summary()

/app/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2023-08-29 06:27:46.834 | DEBUG    | src.services.facebook_image_descriptive_saver:add_global_centroids:128 - len(basic_colors) = 12
100%|██████████| 11569/11569 [46:15<00:00,  4.17it/s] 


shop_id      spend_USD  \
year_month feature      feature_value                               
2021-08    color        #191717         9172502812   40586.106680   
                        #358cc6         9172502812    7689.022054   
                        #363d46         9172502812   40845.956390   
                        #6d6660         9172502812   69281.100676   
                        #7e1dc1         9172502812     627.622295   
...                                            ...            ...   
2023-07    color        #cd8858        15392941704   16498.441533   
                        #e144ad        15392941704    1090.991084   
                        #e7e9eb        15392941704   42664.722603   
                        #f8e330        15392941704    1266.445072   
           fake_feature .              15392941704  283267.254899   

                                               impr         clicks  \
year_month feature      feature_value                                
2021-08    color        #191717        3.438657e+06   55616.669221   
                        #358cc6        1.779994e+06   15387.599792   
                        #363d46        4.351606e+06   51151.878159   
                        #6d6660        6.743652e+06   66614.836360   
                        #7e1dc1        3.449258e+04     274.372166   
...                                             ...            ...   
2023-07    color        #cd8858        3.939907e+06   41147.548640   
                        #e144ad        2.069001e+05    2270.793080   
                        #e7e9eb        8.464770e+06   74189.825192   
                        #f8e330        5.598564e+05   11866.292524   
           fake_feature .              5.688816e+07  913769.000000   

                                             purch  purch_value_USD  \
year_month feature      feature_value                                 
2021-08    color        #191717        1236.469879     71732.369408   
                        #358cc6         179.082381     17750.120056   
                        #363d46         918.852467     67704.019129   
                        #6d6660        1245.143129     84273.414822   
                        #7e1dc1           8.067162       387.442187   
...                                            ...              ...   
2023-07    color        #cd8858         621.321427     62427.829389   
                        #e144ad          29.892749      1846.369704   
                        #e7e9eb        1350.389776    172635.366825   
                        #f8e330         118.112490      6767.248091   
           fake_feature .              8863.000000    940026.027576   

                                             n_ads  n_shops  \
year_month feature      feature_value                         
2021-08    color        #191717         351.167179      167   
                        #358cc6         118.306384      167   
                        #363d46         314.040338      167   
                        #6d6660         383.970632      167   
                        #7e1dc1           2.549826      167   
...                                            ...      ...   
2023-07    color        #cd8858         141.783495      201   
                        #e144ad           8.007943      201   
                        #e7e9eb         514.195129      201   
                        #f8e330          16.556185      201   
           fake_feature .              2069.000000      201   

                                       spend_USD_by_shop  impr_by_shop  \
year_month feature      feature_value                                    
2021-08    color        #191717                 0.074043      0.073086   
                        #358cc6                 0.023048      0.023836   
                        #363d46                 0.094428      0.093820   
                        #6d6660                 0.123360      0.123134   
                        #7e1dc1               

In [2]:
fi.create_and_save_main()

In [2]:
fi.create_and_save_summary()

shop_id     spend_USD          impr  \
year_month feature      feature_value                                         
2021-12    color        #1bcbc8           16038      0.000000  0.000000e+00   
                        #205245           16038      8.839803  3.131108e+02   
                        #27241d           16038    574.587215  2.035220e+04   
                        #536a6a           16038     38.895135  1.377687e+03   
                        #56502f           16038    394.255228  1.396474e+04   
...                                         ...           ...           ...   
2023-07    color        #bbbcb1        44317434  13671.634058  9.567188e+05   
                        #d23024        44317434      0.000000  0.000000e+00   
                        #dcb27f        44317434    391.239130  2.737828e+04   
                        #e9e7df        44317434    282.561594  1.977320e+04   
           fake_feature .              44317434  20996.500000  1.469301e+06   

                                             clicks       purch  \
year_month feature      feature_value                             
2021-12    color        #1bcbc8            0.000000    0.000000   
                        #205245            3.618012    0.025880   
                        #27241d          235.170807    1.682195   
                        #536a6a           15.919255    0.113872   
                        #56502f          161.363354    1.154244   
...                                             ...         ...   
2023-07    color        #bbbcb1        11334.371636  115.902692   
                        #d23024            0.000000    0.000000   
                        #dcb27f          324.354037    3.316770   
                        #e9e7df          234.255694    2.395445   
           fake_feature .              17407.000000  178.000000   

                                       purch_value_USD     n_ads  n_shops  \
year_month feature      feature_value                                       
2021-12    color        #1bcbc8               0.000000  0.000000        1   
                        #205245               5.383540  0.010352        1   
                        #27241d             349.930124  0.672878        1   
                        #536a6a              23.687578  0.045549        1   
                        #56502f             240.105901  0.461698        1   
...                                                ...       ...      ...   
2023-07    color        #bbbcb1           23037.613354  0.651139        2   
                        #d23024               0.000000  0.000000        2   
                        #dcb27f             659.263975  0.155990        2   
                        #e9e7df             476.135093  0.015539        2   
           fake_feature .                 35380.500000  3.000000        2   

                                       spend_USD_by_shop  impr_by_shop  \
year_month feature      feature_value                                    
2021-12    color        #1bcbc8                 0.000000      0.000000   
                        #205245                 0.005176      0.005176   
                        #27241d                 0.336439      0.336439   
                        #536a6a                 0.022774      0.022774   
                        #56502f                 0.230849      0.230849   
...                                                  ...           ...   
2023-07    color        #bbbcb1                 0.651139      0.651139   
                        #d23024                 0.000000      0.000000   
                        #dcb27f                 0.018634      0.018634   
                        #e9e7df                 0.013458      0.013458   
           fake_feature .                       1.000000      1.000000   

                                       clicks_by_shop  purch_by_shop  \
year_month feature      feature_value                                  
2021-12    color        #1bcbc8

In [3]:
df = fi.read_df(FacebookImageDescriptiveDF.main)
df.feature.unique()

array(['color', 'fake_feature'], dtype=object)

In [10]:
performance_df = fi.read_df(df_type=FacebookImageDescriptiveDF.performance)
image_df = fi.read_df(df_type=FacebookImageDescriptiveDF.image)
full_df = performance_df.merge(image_df, on=['url'])
full_df['n_ads'] = 1

In [11]:
fake_feature_df = full_df.groupby(['shop_id', 'year_month'])[fi.metric_columns].sum().reset_index()
fake_feature_df['feature'] = fi.fake_feature
fake_feature_df['feature_value'] = '.'
fake_feature_df.head(5)

,shop_id,year_month,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature_value,feature
0,16038,2021-12,1707.85,60493.0,699.0,5.0,1040.10,2,.,fake_feature
1,16038,2022-01,394.52,8583.0,54.0,11.0,3325.30,1,.,fake_feature
2,16038,2022-02,2276.99,57337.0,235.0,57.0,9727.10,1,.,fake_feature
3,16038,2022-03,3610.73,113389.0,240.0,100.0,15005.20,1,.,fake_feature
4,16038,2022-04,2389.43,66796.0,140.0,101.0,14815.38,1,.,fake_feature


In [2]:
fi.create_and_save_full()

In [3]:
fi.create_and_save_summary()

In [5]:
if not False:
    print('hej')

hej


In [2]:
fi.save_everything(force_from_scratch=True)

2023-08-24 11:37:46.775 | DEBUG    | src.services.descriptive_saver:create_and_save_main:71 - list_of_objects_on_s3 = ['dev/descriptive/facebook_image/2023-07-31_main.csv']
100%|██████████| 2/2 [00:08<00:00,  4.01s/it]
2023-08-24 11:37:57.745 | DEBUG    | src.services.facebook_image_descriptive_saver:update_image_df:83 - ['dev/descriptive/facebook_image/2023-07-31_image.csv']
2023-08-24 11:37:58.866 | DEBUG    | src.services.facebook_image_descriptive_saver:update_image_df:98 - old: 0, new: 14
100%|██████████| 14/14 [00:02<00:00,  5.41it/s]
/app/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2023-08-24 11:38:21.756 | DEBUG    | src.services.facebook_image_descriptive_saver:add_global_centroids:136 - len(basic_colors) = 12
100%|██████████| 14/14 [00:02<00:00,  6.47it/s]


In [2]:
fi.filter_image_df()

In [3]:
fi.create_and_save_performance(db=db, force_from_scratch=True)
fi.filter_and_save_top_n_ads_per_shop_and_month_by_spend()
fi.update_image_df(force_from_scratch=True)
fi.save_ad_images_to_s3(force_from_scratch=True)
fi.add_local_centroids(force_from_scratch=True)
fi.filter_image_df()
fi.add_global_centroids(force_from_scratch=True)
fi.create_and_save_main()
fi.create_and_save_summary()

2023-08-24 12:55:02.154 | DEBUG    | src.services.descriptive_saver:create_and_save_main:72 - list_of_objects_on_s3 = []
100%|██████████| 2/2 [00:23<00:00, 11.50s/it]
2023-08-24 12:55:27.724 | DEBUG    | src.services.facebook_image_descriptive_saver:update_image_df:77 - ['dev/descriptive/facebook_image/2023-07-31_image.csv']
2023-08-24 12:55:29.048 | DEBUG    | src.services.facebook_image_descriptive_saver:update_image_df:92 - old: 0, new: 14
100%|██████████| 14/14 [00:02<00:00,  5.52it/s]
/app/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2023-08-24 12:55:51.980 | DEBUG    | src.services.facebook_image_descriptive_saver:add_global_centroids:130 - len(basic_colors) = 12
100%|██████████| 14/14 [00:02<00:00,  6.76it/s]


AttributeError: full

In [3]:
fi.create_and_save_main()
fi.create_and_save_summary(end_date=fi.end_date)

shop_id     spend_USD           impr  \
year_month feature feature_value                                          
2021-12    color   #1bcbc8           16038      0.000000       0.000000   
                   #205245           16038      8.839803     313.110766   
                   #27241d           16038    574.587215   20352.199793   
                   #536a6a           16038     38.895135    1377.687371   
                   #56502f           16038    394.255228   13964.740166   
...                                    ...           ...            ...   
2023-07    color   #b49f84        44317434   1521.485507  106471.086957   
                   #bbbcb1        44317434  13671.634058  956718.767081   
                   #d23024        44317434      0.000000       0.000000   
                   #dcb27f        44317434    391.239130   27378.279503   
                   #e9e7df        44317434    282.561594   19773.201863   

                                        clicks       purch  purch_value_USD  \
year_month feature feature_value                                              
2021-12    color   #1bcbc8            0.000000    0.000000         0.000000   
                   #205245            3.618012    0.025880         5.383540   
                   #27241d          235.170807    1.682195       349.930124   
                   #536a6a           15.919255    0.113872        23.687578   
                   #56502f          161.363354    1.154244       240.105901   
...                                        ...         ...              ...   
2023-07    color   #b49f84         1261.376812   12.898551      2563.804348   
                   #bbbcb1        11334.371636  115.902692     23037.613354   
                   #d23024            0.000000    0.000000         0.000000   
                   #dcb27f          324.354037    3.316770       659.263975   
                   #e9e7df          234.255694    2.395445       476.135093   

                                     n_ads  n_shops  spend_USD_by_shop  \
year_month feature feature_value                                         
2021-12    color   #1bcbc8        0.000000        1           0.000000   
                   #205245        0.010352        1           0.005176   
                   #27241d        0.672878        1           0.336439   
                   #536a6a        0.045549        1           0.022774   
                   #56502f        0.461698        1           0.230849   
...                                    ...      ...                ...   
2023-07    color   #b49f84        0.159873        2           0.072464   
                   #bbbcb1        0.651139        2           0.651139   
                   #d23024        0.000000        2           0.000000   
                   #dcb27f        0.155990        2           0.018634   
                   #e9e7df        0.015539        2           0.013458   

                                  impr_by_shop  clicks_by_shop  purch_by_shop  \
year_month feature feature_value                                                
2021-12    color   #1bcbc8            0.000000        0.000000       0.000000   
                   #205245            0.005176        0.005176       0.005176   
                   #27241d            0.336439        0.336439       0.336439   
                   #536a6a            0.022774        0.022774       0.022774   
                   #56502f            0.230849        0.230849       0.230849   
...                                        ...             ...            ...   
2023-07    color   #b49f84            0.072464        0.072464       0.072464   
                   #bbbcb1            0.651139        0.651139       0.651139   
                   #d23024            0.000000        0.000000       0.000000   
                   #dcb27f            0.018634        0.018634       0.018634   
                   #e9e7df            0.013458        0.013458       0.013458   

              

In [3]:
fi.save_final()

In [4]:
summary = fi.read_df(df_type=FacebookImageDescriptiveDF.summary, end_date=fi.end_date)

In [5]:
summary

,Unnamed: 0,clicks,relative_impr,spend_USD,relative_clicks,purch,relative_purch_value_USD,relative_spend_USD,index,relative_purch,purch_value_USD,impr,feature
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,color
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12,0.000000,0.000000,0.000000,color
2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,252,0.000000,0.000000,0.000000,color
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36,0.000000,0.000000,0.000000,color
4,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48,0.000000,0.000000,0.000000,color
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,1060.570346,0.606982,1301.249381,0.638173,45.504701,0.785853,0.485912,442,0.772812,3372.628542,127948.863334,color
236,236,558.011446,0.194017,801.595726,0.208602,17.906348,0.197012,0.217664,466,0.203481,1455.541960,138121.718002,color
237,237,219.149688,0.031893,300.011760,0.029030,5.759618,0.020533,0.030158,490,0.015885,782.877062,35620.062151,color
238,238,1.083247,0.001041,1.261595,0.001041,0.043704,0.001041,0.001041,287,0.001041,3.726523,303.913632,color


In [8]:
full_df = fi.read_df(df_type=FacebookImageDescriptiveDF.full, end_date=fi.end_date)

In [9]:
summary_df = full_df.reset_index().groupby(["color", "year_month"]).sum().drop(columns=["shop_id"]).reset_index()
summary_df = (
    summary_df.set_index(list(set(summary_df.columns) - set(["color"])))
    .reset_index()
    .rename(columns={"color": "feature_value"})
    .assign(feature="color")
)

In [10]:
summary_df

,clicks,relative_impr,year_month,spend_USD,relative_clicks,purch,relative_purch_value_USD,relative_spend_USD,index,relative_purch,purch_value_USD,impr,feature_value,feature
0,0.000000,0.000000,2021-12,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,#1acbc8,color
1,0.000000,0.000000,2022-01,0.000000,0.000000,0.000000,0.000000,0.000000,12,0.000000,0.000000,0.000000,#1acbc8,color
2,0.000000,0.000000,2022-02,0.000000,0.000000,0.000000,0.000000,0.000000,252,0.000000,0.000000,0.000000,#1acbc8,color
3,0.000000,0.000000,2022-03,0.000000,0.000000,0.000000,0.000000,0.000000,36,0.000000,0.000000,0.000000,#1acbc8,color
4,0.000000,0.000000,2022-04,0.000000,0.000000,0.000000,0.000000,0.000000,48,0.000000,0.000000,0.000000,#1acbc8,color
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1060.570346,0.606982,2023-03,1301.249381,0.638173,45.504701,0.785853,0.485912,442,0.772812,3372.628542,127948.863334,#e9e7df,color
236,558.011446,0.194017,2023-04,801.595726,0.208602,17.906348,0.197012,0.217664,466,0.203481,1455.541960,138121.718002,#e9e7df,color
237,219.149688,0.031893,2023-05,300.011760,0.029030,5.759618,0.020533,0.030158,490,0.015885,782.877062,35620.062151,#e9e7df,color
238,1.083247,0.001041,2023-06,1.261595,0.001041,0.043704,0.001041,0.001041,287,0.001041,3.726523,303.913632,#e9e7df,color


In [28]:
full_df = fi.read_df(df_type=FacebookImageDescriptiveDF.full, end_date=fi.end_date)

In [24]:
grouped_by_month_color_and_shop = full_df.groupby(["color", "year_month", "shop_id"]).sum().reset_index()

In [25]:
(grouped_by_month_color_and_shop.set_index(list(set(grouped_by_month_color_and_shop.columns) - set(["color"])))
.reset_index()
.rename(columns={"color": "feature_value"})
.assign(feature="color")
)

,clicks,relative_impr,spend_USD,year_month,relative_clicks,purch,relative_purch_value_USD,relative_spend_USD,relative_purch,shop_id,purch_value_USD,impr,feature_value,feature
0,0.000000,0.000000,0.000000,2021-12,0.000000,0.000000,0.000000,0.000000,0.000000,16038,0.000000,0.000000,#1acbc8,color
1,0.000000,0.000000,0.000000,2022-01,0.000000,0.000000,0.000000,0.000000,0.000000,16038,0.000000,0.000000,#1acbc8,color
2,0.000000,0.000000,0.000000,2022-02,0.000000,0.000000,0.000000,0.000000,0.000000,16038,0.000000,0.000000,#1acbc8,color
3,0.000000,0.000000,0.000000,2022-02,0.000000,0.000000,0.000000,0.000000,0.000000,44301396,0.000000,0.000000,#1acbc8,color
4,0.000000,0.000000,0.000000,2022-03,0.000000,0.000000,0.000000,0.000000,0.000000,16038,0.000000,0.000000,#1acbc8,color
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,139.030021,0.007409,173.072930,2023-05,0.008777,1.601449,0.008486,0.007192,0.008564,16038,315.108696,12997.254658,#e9e7df,color
296,80.119667,0.024483,126.938830,2023-05,0.020253,4.158169,0.012047,0.022966,0.007321,44301396,467.768366,22622.807492,#e9e7df,color
297,1.083247,0.001041,1.261595,2023-06,0.001041,0.043704,0.001041,0.001041,0.001041,44301396,3.726523,303.913632,#e9e7df,color
298,234.255694,0.013458,282.561594,2023-07,0.013458,2.395445,0.013458,0.013458,0.013458,16038,476.135093,19773.201863,#e9e7df,color


In [26]:
grouped_by_month_color_and_shop.set_index(list(set(grouped_by_month_color_and_shop.columns) - set(["color"])))

color
clicks     relative_impr spend_USD  year_month relative_clicks purch    relative_purch_value_USD relative_spend_USD relative_purch shop_id  purch_value_USD impr                 
0.000000   0.000000      0.000000   2021-12    0.000000        0.000000 0.000000                 0.000000           0.000000       16038    0.000000        0.000000      #1acbc8
                                    2022-01    0.000000        0.000000 0.000000                 0.000000           0.000000       16038    0.000000        0.000000      #1acbc8
                                    2022-02    0.000000        0.000000 0.000000                 0.000000           0.000000       16038    0.000000        0.000000      #1acbc8
                                                                                                                                   44301396 0.000000        0.000000      #1acbc8
                                    2022-03    0.000000        0.000000 0.000000                 0.000000           0.000000       16038    0.000000        0.000000      #1acbc8
...                                                                                                                                                                           ...
139.030021 0.007409      173.072930 2023-05    0.008777        1.601449 0.008486                 0.007192           0.008564       16038    315.108696      12997.254658  #e9e7df
80.119667  0.024483      126.938830 2023-05    0.020253        4.158169 0.012047                 0.022966           0.007321       44301396 467.768366      22622.807492  #e9e7df
1.083247   0.001041      1.261595   2023-06    0.001041        0.043704 0.001041                 0.001041           0.001041       44301396 3.726523        303.913632    #e9e7df
234.255694 0.013458      282.561594 2023-07    0.013458        2.395445 0.013458                 0.013458           0.013458       16038    476.135093      19773.201863  #e9e7df
0.000000   0.000000      0.000000   2023-07    0.000000        0.000000 0.000000                 0.000000           0.000000       44301396 0.000000        0.000000      #e9e7df

[300 rows x 1 columns]

In [23]:
list(set(grouped_by_month_color_and_shop.columns) - set(["color"]))

['clicks',
 'relative_impr',
 'spend_USD',
 'relative_clicks',
 'purch',
 'relative_purch_value_USD',
 'relative_spend_USD',
 'relative_purch',
 'purch_value_USD',
 'impr']

In [29]:
full_df

,shop_id,year_month,color,spend_USD,relative_spend_USD,impr,relative_impr,clicks,relative_clicks,purch,relative_purch,purch_value_USD,relative_purch_value_USD
0,16038,2021-12,#1acbc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,16038,2021-12,#225041,26.519410,0.015528,939.332298,0.015528,10.854037,0.015528,0.077640,0.015528,16.150621,0.015528
2,16038,2021-12,#27241d,563.979451,0.330228,19976.466874,0.330228,230.829193,0.330228,1.651139,0.330228,343.469876,0.330228
3,16038,2021-12,#356368,8.839803,0.005176,313.110766,0.005176,3.618012,0.005176,0.025880,0.005176,5.383540,0.005176
4,16038,2021-12,#564f2f,390.719306,0.228778,13839.495859,0.228778,159.916149,0.228778,1.143892,0.228778,237.952484,0.228778
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,44301396,2023-07,#a1acbf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
296,44301396,2023-07,#baa183,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
297,44301396,2023-07,#c1c0ad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
298,44301396,2023-07,#d23024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
perf_df = fi.read_df(df_type=FacebookImageDescriptiveDF.main, end_date=fi.end_date)
image_df = fi.read_df(df_type=FacebookImageDescriptiveDF.filtered_image, end_date=fi.end_date)

image_df = image_df[image_df.global_color_centroids.notna()]
image_df.global_color_centroids = image_df.global_color_centroids.apply(
    lambda x: {rgb_to_hex(k): v for k, v in x.items()}
)
unpacked_colors = image_df.global_color_centroids.apply(pd.Series).fillna(0)
scaled_colors = unpacked_colors.div(unpacked_colors.sum(axis=1), axis=0)
image_df = image_df.join(scaled_colors)

full_df = perf_df.merge(image_df, on="url")
full_df = full_df[full_df.shop_id != 37673090]
full_df['n_ads'] = 1
color_cols = [col for col in full_df.columns if col[0] == "#"]
full_df = full_df[['ad_id', 'shop_id', 'year_month']+fi.metric_columns+color_cols]
noncolor_cols = [col for col in full_df.columns if col[0] != "#"]
stacked_df = full_df.set_index(noncolor_cols).stack(level=0).reset_index()
level_col = [col for col in stacked_df.columns if 'level' in str(col)][0]
stacked_df = stacked_df.rename(columns={level_col: 'color', 0: 'freq'})
stacked_df[fi.metric_columns] = stacked_df[fi.metric_columns].multiply(stacked_df.freq, axis=0)
full_stacked_df = stacked_df.rename(columns={'color': 'feature_value'}).drop(columns=['freq']).assign(feature='color')
full_stacked_df = full_stacked_df.groupby(fi.main_df_index + ['shop_id'])[fi.metric_columns].sum()

In [82]:
full_stacked_df

spend_USD          impr  \
year_month feature feature_value shop_id                              
2021-12    color   #1acbc8       16038       0.000000      0.000000   
                   #225041       16038      26.519410    939.332298   
                   #27241d       16038     563.979451  19976.466874   
                   #356368       16038       8.839803    313.110766   
                   #564f2f       16038     390.719306  13839.495859   
...                                               ...           ...   
2023-07    color   #c1c0ad       44301396    0.000000      0.000000   
                   #d23024       16038       0.000000      0.000000   
                                 44301396    0.000000      0.000000   
                   #e9e7df       16038     282.561594  19773.201863   
                                 44301396    0.000000      0.000000   

                                               clicks     purch  \
year_month feature feature_value shop_id                          
2021-12    color   #1acbc8       16038       0.000000  0.000000   
                   #225041       16038      10.854037  0.077640   
                   #27241d       16038     230.829193  1.651139   
                   #356368       16038       3.618012  0.025880   
                   #564f2f       16038     159.916149  1.143892   
...                                               ...       ...   
2023-07    color   #c1c0ad       44301396    0.000000  0.000000   
                   #d23024       16038       0.000000  0.000000   
                                 44301396    0.000000  0.000000   
                   #e9e7df       16038     234.255694  2.395445   
                                 44301396    0.000000  0.000000   

                                           purch_value_USD     n_ads  
year_month feature feature_value shop_id                              
2021-12    color   #1acbc8       16038            0.000000  0.000000  
                   #225041       16038           16.150621  0.031056  
                   #27241d       16038          343.469876  0.660455  
                   #356368       16038            5.383540  0.010352  
                   #564f2f       16038          237.952484  0.457557  
...                                                    ...       ...  
2023-07    color   #c1c0ad       44301396         0.000000  0.043704  
                   #d23024       16038            0.000000  0.000000  
                                 44301396         0.000000  0.000000  
                   #e9e7df       16038          476.135093  0.013458  
                                 44301396         0.000000  0.002081  

[300 rows x 6 columns]

In [71]:
stacked_df = full_df.set_index(noncolor_cols).stack(level=0).reset_index()
level_col = [col for col in stacked_df.columns if 'level' in str(col)][0]
stacked_df = stacked_df.rename(columns={level_col: 'color', 0: 'freq'})

In [73]:
stacked_df[fi.metric_columns] = stacked_df[fi.metric_columns].multiply(stacked_df.freq, axis=0)

In [77]:
stacked_df.rename(columns={'color': 'feature_value'}).drop(columns=['freq']).assign(feature='color')

,ad_id,shop_id,year_month,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature_value,feature
0,23849035741720755,16038,2021-12,103.596273,3731.490683,42.981366,0.186335,41.863354,0.062112,#e9e7df,color
1,23849035741720755,16038,2021-12,124.315528,4477.788820,51.577640,0.223602,50.236025,0.074534,#c1c0ad,color
2,23849035741720755,16038,2021-12,272.803520,9826.258799,113.184265,0.490683,110.240166,0.163561,#8e7853,color
3,23849035741720755,16038,2021-12,162.300828,5846.002070,67.337474,0.291925,65.585921,0.097308,#baa183,color
4,23849035741720755,16038,2021-12,381.579607,13744.324017,158.314700,0.686335,154.196687,0.228778,#564f2f,color
...,...,...,...,...,...,...,...,...,...,...,...
667,6318012909858,44301396,2023-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.227888,#27241d,color
668,6318012909858,44301396,2023-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.016649,#225041,color
669,6318012909858,44301396,2023-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.020812,#356368,color
670,6318012909858,44301396,2023-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.034339,#d23024,color


In [52]:
[col for col in stacked_df.columns]

['ad_id',
 'shop_id',
 'year_month',
 'spend_USD',
 'impr',
 'clicks',
 'purch',
 'purch_value_USD',
 'level_8',
 0]

In [34]:
noncolor_columns = [col for col in full_df.columns if '#' not in col]
color_df = (
    full_df.set_index(noncolor_columns)
    # .stack(level=0)
    # .reset_index()
    # .rename(columns={"level_5": "color", 0: "freq"})
)

TypeError: unhashable type: 'dict'

In [42]:
metrike = [col for col in fi.metric_columns if col != 'n_ads']
full_df = full_df[['ad_id', 'shop_id', 'year_month']+metrike+color_cols]